**Задание 1**

1. Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [4]:
import pandas as pd

In [21]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [35]:
previous_user = None
counter = 0
time_start = 0
time_end = 0
time_spent = []

with open('ml-latest-small/ratings.csv', encoding='utf8') as f:
    f.readline()
    for i, line in enumerate(f):
        userId, movieId, rating,timestamp = line.strip().split(',')
#        print(userId, movieId, rating,timestamp)
        
        if previous_user == userId:
            counter += 1
        else:
            previous_user = userId
            counter = 1
        if counter == 1:
            time_start = timestamp
        if counter == 100:
            time_end = timestamp
            time_spent.append(int(time_end)-int(time_start))
#            print(userId, time_end, time_start)

#            if i > 10:
#            break
sum(time_spent) / len(time_spent)

102237.01520912547

**Задание  2**

2. Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [36]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [37]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [38]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [39]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [47]:
#таблица с тремя типами выручки для каждого client_id без указания адреса клиента
(rzd.merge(auto, how='outer')
 .merge(air, how='outer')
 .fillna(0)
 .reset_index()
)

,index,client_id,rzd_revenue,auto_revenue,air_revenue
0,0,111,1093.0,0.0,0.0
1,1,112,2810.0,0.0,0.0
2,2,113,10283.0,57483.0,0.0
3,3,114,5774.0,83.0,0.0
4,4,115,981.0,912.0,81.0
5,5,116,0.0,4834.0,4.0
6,6,117,0.0,98.0,13.0
7,7,118,0.0,0.0,173.0


In [50]:
#аналогичную таблицу по типам выручки с указанием адреса клиента
(rzd.merge(auto, how='outer')
 .merge(air, how='outer')
 .merge(client_base, how='outer')
 .fillna(0)
 .reset_index()
)

,index,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,0,111,1093.0,0.0,0.0,Комсомольская 4
1,1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,2,113,10283.0,57483.0,0.0,Левобережная 1а
3,3,114,5774.0,83.0,0.0,Мира 14
4,4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,5,116,0.0,4834.0,4.0,Строителей 18
6,6,117,0.0,98.0,13.0,Панфиловская 33
7,7,118,0.0,0.0,173.0,Мастеркова 4


**Задание 3**

3. В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.


Исходя из вводных данных, можно предположить, что для каждой координаты пользователя известно время, когда она была определена.
Для связывания координаты с визитом или фактом покупки можно использовать координату пользователя, соответсвующую или примерно соответствующую (отдаленную на погрешность определения координаты) координате магазина или пункта выдачи товара.

Координаты, связанные со временем и фактом покупки в магазинах может помочь определить, как часто посетители посещают магазин без покупки, и как часто они на нее решаются.

Кроме того, исходи из координат и времени их регистрирования (они становятся более или менее удалены друг от друга) и зная местоположение метро/автобусных остановок, можно сказать, каким типом транспорта чаще всего передвигаются пользователи (быстрее, например, метро чем автобусы и т д) и какие типы товаров они чаще всего покупают в определенных филиалах магазина или пунктах выдачи. Например, пользователи, которые чаще всего ходят в магазин Х пользуются метро и реже покупают объемные товары, ввиду чего они пользуются низким спросом.

Помимо этого, если загруженность магазина/заведения большая в час пик, можно определить, сколько человек проводит в очереди за товаром/услугой исходя из того, сколько он находится в заведении.

В целом, передвижение человека в течение дня может много сказать о пользователе, от образа жизни, способа передвижения, до примерного рода работы, на чем можно основывать аналитику.

В целом таблица бы выглядела так:
пользователь - координата - время - метро (да/нет) - иной транспорт(да-нет) - посещение (да/нет) - покупка (сумма) - длительное ожидание рассчет времени посещения. 

Таким образом у каждого пользователя был бы список координат, отсортированный по времени, и каждую из которых можно связать с фактом посещения, покупки, длительного ожидания, способа перемещения. 

